# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [131]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Generate the mathematical model

In [159]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [160]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Alkylation Process

**Parameters:**

*  `p`: Price per barrel of alkylate
* `c_o`: Cost per barrel of olefin feed
* `c_i`: Cost per barrel of isobutane 
* `c_a`: Cost per thousand pounds of acid addition

**Decision Variables:**

* `OF`: Olefin feed (barrels per day)
* `IR`: Isobutane recycle (barrels per day)
* `AAR`: Acid addition rate (thousands of pounds per day)
* `AY`: Alkylate yield (barrels per day)
* `IM`: Isobutane makeup (barrels per day)
* `AS`: Acid strength (weight percent)
* `MON`: Motor octane number
* `IOR`: External isobutane-to-olefin ratio
* `ADF`: Acid dilution factor
* `F4`: F-4 performance number 

**Objective Function:**

Maximize daily profit:

```
Maximize: p * AY * MON - (c_o * OF + c_i * (IR + IM) + c_a * AAR)
```

**Constraints:**

* **Alkylate Yield Regression:**
   ```
   0.98 * (1.12 + 0.13167 * IOR - 0.00667 * IOR^2) * OF <= AY <= 1.02 * (1.12 + 0.13167 * IOR - 0.00667 * IOR^2) * OF
   ```
* **Motor Octane Number Regression:**
   ```
   0.99 * (86.35 + 1.098 * IOR - 0.038 * IOR^2 + 0.325 * AS - 89) <= MON <= 1.01 * (86.35 + 1.098 * IOR - 0.038 * IOR^2 + 0.325 * AS - 89)
   ```
* **Acid Dilution Factor Regression:**
   ```
   0.95 * (35.82 - 0.222 * F4) <= ADF <= 1.05 * (35.82 - 0.222 * F4)
   ```
* **F-4 Performance Number Regression:**
   ```
   0.9 * (-133 + 3 * MON) <= F4 <= 1.1 * (-133 + 3 * MON)
   ```
* **External Isobutane-to-Olefin Ratio:**
   ```
   IOR = (IR + IM) / OF
   ```
* **Acid Strength:**
   ```
   AS = (93000 * AAR) / (AAR * ADF + 1000 * AAR) 
   ```
* **Alkylate Yield Balance:**
   ```
   1.22 * AY = OF + IM
   ```
* **Variable Bounds:**
   ```
   OF_lower <= OF <= OF_upper
   IR_lower <= IR <= IR_upper
   AAR_lower <= AAR <= AAR_upper
   AY_lower <= AY <= AY_upper
   IM_lower <= IM <= IM_upper
   AS_lower <= AS <= AS_upper
   MON_lower <= MON <= MON_upper
   IOR_lower <= IOR <= IOR_upper
   ADF_lower <= ADF <= ADF_upper
   F4_lower <= F4 <= F4_upper
   ```

This model defines the problem of maximizing the alkylation process's profit by optimizing the ten decision variables. The objective function represents the profit, calculated as revenue minus costs. The constraints capture the relationships between variables based on regression formulas, process requirements, and variable bounds. 

This model can be solved using optimization software to determine the optimal values for each decision variable, maximizing the alkylation process's daily profit. 


## 3. Generate the pyomo code

In [161]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [162]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
p = 100  # Price per barrel of alkylate
c_o = 50  # Cost per barrel of olefin feed
c_i = 40  # Cost per barrel of isobutane
c_a = 2  # Cost per thousand pounds of acid addition

OF_lower, OF_upper = 1000, 5000  # Olefin feed bounds
IR_lower, IR_upper = 500, 2500  # Isobutane recycle bounds
AAR_lower, AAR_upper = 10, 50  # Acid addition rate bounds
AY_lower, AY_upper = 1200, 6000  # Alkylate yield bounds
IM_lower, IM_upper = 100, 500  # Isobutane makeup bounds
AS_lower, AS_upper = 85, 95  # Acid strength bounds
MON_lower, MON_upper = 88, 94  # Motor octane number bounds
IOR_lower, IOR_upper = 3, 6  # External isobutane-to-olefin ratio bounds
ADF_lower, ADF_upper = 20, 40  # Acid dilution factor bounds
F4_lower, F4_upper = 120, 140  # F-4 performance number bounds

# Create a concrete Pyomo model
model = pyo.ConcreteModel()

# Decision Variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IR = pyo.Var(bounds=(IR_lower, IR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IM = pyo.Var(bounds=(IM_lower, IM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Objective Function
model.profit = pyo.Objective(
    expr=p * model.AY * model.MON - (c_o * model.OF + c_i * (model.IR + model.IM) + c_a * model.AAR),
    sense=pyo.maximize)

# Constraints
# Alkylate Yield Regression
model.alkylate_yield_regression = pyo.Constraint(
    expr=0.98 * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF <= model.AY,
    expr=model.AY <= 1.02 * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF)

# Motor Octane Number Regression
model.motor_octane_regression = pyo.Constraint(
    expr=0.99 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89) <= model.MON,
    expr=model.MON <= 1.01 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89))

# Acid Dilution Factor Regression
model.acid_dilution_regression = pyo.Constraint(
    expr=0.95 * (35.82 - 0.222 * model.F4) <= model.ADF,
    expr=model.ADF <= 1.05 * (35.82 - 0.222 * model.F4))

# F-4 Performance Number Regression
model.f4_performance_regression = pyo.Constraint(
    expr=0.9 * (-133 + 3 * model.MON) <= model.F4,
    expr=model.F4 <= 1.1 * (-133 + 3 * model.MON))

# External Isobutane-to-Olefin Ratio
model.external_isobutane_ratio = pyo.Constraint(expr=model.IOR == (model.IR + model.IM) / model.OF)

# Acid Strength
model.acid_strength = pyo.Constraint(expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))

# Alkylate Yield Balance
model.alkylate_yield_balance = pyo.Constraint(expr=1.22 * model.AY == model.OF + model.IM)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can use other solvers like 'glpk', 'cbc', etc.
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"OF = {pyo.value(model.OF)} barrels per day")
print(f"IR = {pyo.value(model.IR)} barrels per day")
print(f"AAR = {pyo.value(model.AAR)} thousand pounds per day")
print(f"AY = {pyo.value(model.AY)} barrels per day")
print(f"IM = {pyo.value(model.IM)} barrels per day")
print(f"AS = {pyo.value(model.AS)} weight percent")
print(f"MON = {pyo.value(model.MON)}")
print(f"IOR = {pyo.value(model.IOR)}")
print(f"ADF = {pyo.value(model.ADF)}")
print(f"F4 = {pyo.value(model.F4)}")
print(f"Profit = ${pyo.value(model.profit)} per day")
```

This code defines the Pyomo model for the alkylation process optimization problem. Note that the sample data is indicated with comments and should be replaced with actual data for meaningful results. You can use a suitable solver like 'ipopt', 'glpk', or 'cbc' to solve the model and obtain the optimal values for the decision variables. 


## 4. Input problem data and try running the generated code

In [164]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data
p = 100  # Price per barrel of alkylate
c_o = 50  # Cost per barrel of olefin feed
c_i = 40  # Cost per barrel of isobutane
c_a = 2  # Cost per thousand pounds of acid addition

OF_lower, OF_upper = 1000, 5000  # Olefin feed bounds
IR_lower, IR_upper = 500, 2500  # Isobutane recycle bounds
AAR_lower, AAR_upper = 10, 50  # Acid addition rate bounds
AY_lower, AY_upper = 1200, 6000  # Alkylate yield bounds
IM_lower, IM_upper = 100, 500  # Isobutane makeup bounds
AS_lower, AS_upper = 85, 95  # Acid strength bounds
MON_lower, MON_upper = 88, 94  # Motor octane number bounds
IOR_lower, IOR_upper = 3, 6  # External isobutane-to-olefin ratio bounds
ADF_lower, ADF_upper = 20, 40  # Acid dilution factor bounds
F4_lower, F4_upper = 120, 140  # F-4 performance number bounds

# Create a concrete Pyomo model
model = pyo.ConcreteModel()

# Decision Variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IR = pyo.Var(bounds=(IR_lower, IR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IM = pyo.Var(bounds=(IM_lower, IM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Objective Function
model.profit = pyo.Objective(
    expr=p * model.AY * model.MON - (c_o * model.OF + c_i * (model.IR + model.IM) + c_a * model.AAR),
    sense=pyo.maximize)

# Constraints
# Alkylate Yield Regression
model.alkylate_yield_regression = pyo.Constraint(
    expr=0.98 * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF <= model.AY,
    expr=model.AY <= 1.02 * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF)

# Motor Octane Number Regression
model.motor_octane_regression = pyo.Constraint(
    expr=0.99 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89) <= model.MON,
    expr=model.MON <= 1.01 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89))

# Acid Dilution Factor Regression
model.acid_dilution_regression = pyo.Constraint(
    expr=0.95 * (35.82 - 0.222 * model.F4) <= model.ADF,
    expr=model.ADF <= 1.05 * (35.82 - 0.222 * model.F4))

# F-4 Performance Number Regression
model.f4_performance_regression = pyo.Constraint(
    expr=0.9 * (-133 + 3 * model.MON) <= model.F4,
    expr=model.F4 <= 1.1 * (-133 + 3 * model.MON))

# External Isobutane-to-Olefin Ratio
model.external_isobutane_ratio = pyo.Constraint(expr=model.IOR == (model.IR + model.IM) / model.OF)

# Acid Strength
model.acid_strength = pyo.Constraint(expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))

# Alkylate Yield Balance
model.alkylate_yield_balance = pyo.Constraint(expr=1.22 * model.AY == model.OF + model.IM)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can use other solvers like 'glpk', 'cbc', etc.
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"OF = {pyo.value(model.OF)} barrels per day")
print(f"IR = {pyo.value(model.IR)} barrels per day")
print(f"AAR = {pyo.value(model.AAR)} thousand pounds per day")
print(f"AY = {pyo.value(model.AY)} barrels per day")
print(f"IM = {pyo.value(model.IM)} barrels per day")
print(f"AS = {pyo.value(model.AS)} weight percent")
print(f"MON = {pyo.value(model.MON)}")
print(f"IOR = {pyo.value(model.IOR)}")
print(f"ADF = {pyo.value(model.ADF)}")
print(f"F4 = {pyo.value(model.F4)}")
print(f"Profit = ${pyo.value(model.profit)} per day")

SyntaxError: keyword argument repeated: expr (3513691711.py, line 44)

## 5. Correct the code to verify model viability (optional)

In [166]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data
p = 100  # Price per barrel of alkylate
c_o = 50  # Cost per barrel of olefin feed
c_i = 40  # Cost per barrel of isobutane
c_a = 2  # Cost per thousand pounds of acid addition

OF_lower, OF_upper = 1000, 5000  # Olefin feed bounds
IR_lower, IR_upper = 500, 2500  # Isobutane recycle bounds
AAR_lower, AAR_upper = 10, 50  # Acid addition rate bounds
AY_lower, AY_upper = 1200, 6000  # Alkylate yield bounds
IM_lower, IM_upper = 100, 500  # Isobutane makeup bounds
AS_lower, AS_upper = 85, 95  # Acid strength bounds
MON_lower, MON_upper = 88, 94  # Motor octane number bounds
IOR_lower, IOR_upper = 3, 6  # External isobutane-to-olefin ratio bounds
ADF_lower, ADF_upper = 20, 40  # Acid dilution factor bounds
F4_lower, F4_upper = 120, 140  # F-4 performance number bounds

# Create a concrete Pyomo model
model = pyo.ConcreteModel()

# Decision Variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IR = pyo.Var(bounds=(IR_lower, IR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IM = pyo.Var(bounds=(IM_lower, IM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Objective Function
model.profit = pyo.Objective(
    expr=p * model.AY * model.MON - (c_o * model.OF + c_i * (model.IR + model.IM) + c_a * model.AAR),
    sense=pyo.maximize)

# Constraints
# Alkylate Yield Regression
model.alkylate_yield_regression = pyo.Constraint(
    expr=model.AY <= 1.02 * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF)
model.alkylate_yield_regression2 = pyo.Constraint(
    expr=0.98 * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF <= model.AY)

# Motor Octane Number Regression
model.motor_octane_regression = pyo.Constraint(
    
    expr=model.MON <= 1.01 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89))
model.motor_octane_regression2= pyo.Constraint(
    expr=0.99 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89) <= model.MON)

# Acid Dilution Factor Regression
model.acid_dilution_regression = pyo.Constraint(
    expr=model.ADF <= 1.05 * (35.82 - 0.222 * model.F4))
    
model.acid_dilution_regression2= pyo.Constraint(
    expr=0.95 * (35.82 - 0.222 * model.F4) <= model.ADF)


# F-4 Performance Number Regression
model.f4_performance_regression = pyo.Constraint(
    expr=model.F4 <= 1.1 * (-133 + 3 * model.MON))
    
# F-4 Performance Number Regression
model.f4_performance_regression2= pyo.Constraint(
    expr=0.9 * (-133 + 3 * model.MON) <= model.F4)

# External Isobutane-to-Olefin Ratio
model.external_isobutane_ratio = pyo.Constraint(expr=model.IOR == (model.IR + model.IM) / model.OF)

# Acid Strength
model.acid_strength = pyo.Constraint(expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))

# Alkylate Yield Balance
model.alkylate_yield_balance = pyo.Constraint(expr=1.22 * model.AY == model.OF + model.IM)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can use other solvers like 'glpk', 'cbc', etc.
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"OF = {pyo.value(model.OF)} barrels per day")
print(f"IR = {pyo.value(model.IR)} barrels per day")
print(f"AAR = {pyo.value(model.AAR)} thousand pounds per day")
print(f"AY = {pyo.value(model.AY)} barrels per day")
print(f"IM = {pyo.value(model.IM)} barrels per day")
print(f"AS = {pyo.value(model.AS)} weight percent")
print(f"MON = {pyo.value(model.MON)}")
print(f"IOR = {pyo.value(model.IOR)}")
print(f"ADF = {pyo.value(model.ADF)}")
print(f"F4 = {pyo.value(model.F4)}")
print(f"Profit = ${pyo.value(model.profit)} per day")

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.11.1\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
Optimal Solution:
OF = 1000.0 barrels per day
IR = 2499.999935000016 barrels per day
AAR = 10.31528480314869 thousand pounds per day
AY = 1229.5081926229548 barrels per day
IM = 500.0 barrels per day
AS = 91.17647060611363 weight percent
MON = 88.0
IOR = 3.0
ADF = 20.0
F4 = 120.0
Profit = $10649651.467112394 per day


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [157]:
print(response.text)

## Mathematical Optimization Model for Alkylation Process

**Parameters:**

* **Cost_Olefin:** Cost per barrel of olefin feed.
* **Cost_Isobutane:** Cost per barrel of isobutane.
* **Cost_Acid:** Cost per thousand pounds of acid.
* **Revenue_Alkylate:** Revenue per barrel of alkylate.

**Decision Variables:**

* **Olefin_Feed:** Olefin feed (barrels per day).
* **Isobutane_Recycle:** Isobutane recycle (barrels per day).
* **Acid_Addition_Rate:** Acid addition rate (thousands of pounds per day).
* **Alkylate_Yield:** Alkylate yield (barrels per day).
* **Isobutane_Makeup:** Isobutane makeup (barrels per day).
* **Acid_Strength:** Acid strength (weight percent).
* **Motor_Octane_Number:** Motor octane number.
* **Ext_Isobutane_Olefin_Ratio:** External isobutane-to-olefin ratio.
* **Acid_Dilution_Factor:** Acid dilution factor.
* **F4_Performance_Number:** F-4 performance number.

**Objective Function:**

Maximize daily profit:

```
Profit = Revenue_Alkylate * Alkylate_Yield * Motor_Octan

In [158]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data if available)
Cost_Olefin = 50  # $/barrel
Cost_Isobutane = 45  # $/barrel
Cost_Acid = 100  # $/thousand pounds
Revenue_Alkylate = 70  # $/barrel

# Model Initialization
model = pyo.ConcreteModel()

# Decision Variables
model.Olefin_Feed = pyo.Var(bounds=(1000, 5000), doc="Olefin feed (barrels per day)") # Sample bounds
model.Isobutane_Recycle = pyo.Var(bounds=(500, 2500), doc="Isobutane recycle (barrels per day)") # Sample bounds
model.Acid_Addition_Rate = pyo.Var(bounds=(10, 50), doc="Acid addition rate (thousands of pounds per day)") # Sample bounds
model.Alkylate_Yield = pyo.Var(bounds=(1000, 6000), doc="Alkylate yield (barrels per day)") # Sample bounds
model.Isobutane_Makeup = pyo.Var(bounds=(100, 500), doc="Isobutane makeup (barrels per day)") # Sample bounds
model.Acid_Strength = pyo.Var(bounds=(88, 92), doc="Acid strength (weight percent)") # Sample bounds
model.Motor_Octane_Number = pyo.Var(bounds=